In [102]:
from PIL import Image, ImageDraw, ImageFont
from pathlib import Path
from pprint import pp

tile_size = 8
font_path = r'..\..\resources\胡晓波男神体2.0.otf'
palette_path = Path('pal1_00-9C-BD-FF.png')
optimal_font_size = []

palette = None
with Image.open(palette_path) as im:
    palette = tuple(entry[1] for entry in im.convert('L').getcolors())

pp(palette)

(0, 156, 189, 255)


In [103]:
def real_bbox(text, font_path: str, font_size: int) -> Image:
    lines = tuple(text)
    width = max([len(line) for line in lines]) * font_size
    height = len(lines) * font_size
    with Image.new('L', (width*3, height*3)) as img:
        draw = ImageDraw.Draw(img)
        font = ImageFont.truetype(font_path, font_size)
        draw.text(
            (width, height), "\n".join(lines), font=font, fill="white",
            align="center", spacing=-0.7,
            stroke_width=0.2, stroke_fill=192
        )
        
        xMin = width * 3
        yMin = height * 3
        xMax, yMax = 0, 0
        for y in range(img.height):
            for x in range(img.width):
                if (v := img.getpixel((x, y))) not in palette:
                    v = min(palette, key=lambda c: abs(c**2-v**2))
                    img.putpixel((x, y), v)
                    
                if v != 0:
                    xMin = min(xMin, x)
                    yMin = min(yMin, y)
                    xMax = max(xMax, x)
                    yMax = max(yMax, y)

        return img.crop((xMin, yMin, xMax+1, yMax+1))

In [104]:
def generate_text_img(text: str, max_columns: int, max_rows: int, font_path: str) -> Image:
    global optimal_font_size
    max_width = int(max_columns * tile_size)
    max_height = int(max_rows * tile_size)
    font_size = max(max_width, max_height)
    while True:
        img = real_bbox(text, font_path, font_size)
        if img.height > max_height or img.width > max_width:
            font_size -= 1
        else:
            print(f'{font_size = }')
            optimal_font_size.append(font_size)
            break

    print(f'width = {img.width}, height = {img.height}')
    print(f'{img.getcolors() = }')
    return img

In [105]:
text = "女高中生"
max_columns = 2.5
max_rows = 9

generate_text_img(text, max_columns, max_rows, font_path)

font_size = 17
width = 17, height = 69
img.getcolors() = [(646, 0), (32, 156), (97, 189), (398, 255)]


In [106]:
text = "连续绑架事件"
max_columns = 2.5
max_rows = 14

generate_text_img(text, max_columns, max_rows, font_path)

font_size = 18
width = 18, height = 111
img.getcolors() = [(841, 0), (173, 156), (271, 189), (713, 255)]


In [107]:
font_size = min(optimal_font_size)
print(f'{optimal_font_size = }')

im_1 = real_bbox("女高中生", font_path, font_size)
im_2 = real_bbox("连续绑架事件", font_path, font_size)

with Image.open("0F0000.png", 'r') as bg:
    bg.paste((0,0,0), (176,160,216,272))
    bg.paste(im_2, (176, 272-im_2.height))
    bg.paste(im_1, (216-im_1.width, 160))
    bg.save("0F0000_chs.png")


optimal_font_size = [17, 18]


In [108]:
import subprocess

result = subprocess.run('encodegfx.bat', capture_output=True, text=True)
print(result.stdout)


(ws-conan-1-chs) d:\Users\John\Code\WS_Conan_1_CHS\graphic\gfx_05CE24>tool\PixelPet\PixelPet.exe     Import-Bitmap "graphic\gfx_05CE24\pal0_00-73-42-DE.png" Read-Palettes --palette-number 0 --palette-size 4     Convert-Palettes 32BPP     Import-Bitmap "graphic\gfx_05CE24\0F0000_chs.png" --format 32BPP     Generate-Tilemap WS --height 144    Import-Bitmap "graphic\gfx_05CE24\pal5_00-9C-BD-FF.png" Read-Palettes --palette-number 5 --palette-size 4     Import-Bitmap "graphic\gfx_05CE24\pal1_00-9C-BD-FF.png" Read-Palettes --palette-number 1 --palette-size 4 --append     Convert-Palettes 32BPP     Import-Bitmap "graphic\gfx_05CE24\0F0000_chs.png" --format 32BPP     Generate-Tilemap WS --append --y 144    Serialize-Tileset Export-Bytes "graphic\gfx_05CE24\0F07F0-0F209F_chs.2bpp"     Serialize-Tilemap Export-Bytes "graphic\gfx_05CE24\0F0000_chs.tilemap"       || exit /b 1 

